In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('./data/final_data.csv')

/var/folders/bk/03p2dw31415ct52_2rx56fmm0000gn/T/ipykernel_7601/3926277462.py:1: DtypeWarning: Columns (18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/final_data.csv')


In [ ]:
# from dataprep.eda import create_report

# # data prep report 생성 및 보기
# classic_report = create_report(df)
# classic_report.save('classic_final_report') # save report to local disk
# classic_report.show_browser()

In [ ]:
df.head(1)

,연령,성별,예매일자,예매시간,공연일자,공연시작시간,좌석,결제가격,취소여부,할인내역,선예매시작일,예매시작일,장르,휴게시간,회원여부,공연명,내한여부,출연진,제작진,러닝타임,관람연령,제작사,가격종류,R석,S석,A석,티켓판매량,요일,멤버십,무료,싹틔우미,노블,골드,블루,그린,층,블록,열,좌석번호,멤버십점수,멤버십등급
0,50.0,M,20220206,1615,2022-03-02,1930,1층 B블록12열 7,180000,0,일반,20220109.0,20220110.0,독주,0,Y,"국립합창단 위대한 합창 Ⅰ, 칼 오르프 & 카르미나 부라나",0,"윤의중, 박미자, 박의준, 한명원 등",NaN,1시간 10분,만 7세 이상,NaN,"[30000, 20000, 10000]",30000.0,20000.0,10000.0,1039,2,무료,1,0,0,0,0,0,1,B,12,7,1,D


In [ ]:
df.멤버십점수.value_counts().sort_index()

0     442260
1      67162
3      37974
4      48496
5      25712
6      82104
7       4005
8      19638
9      17967
10     31620
11      2921
12      3097
13     10097
15       860
Name: 멤버십점수, dtype: int64

In [ ]:
membership_score = df.멤버십.value_counts(dropna=False).sort_values(ascending=False)
membership_score = pd.DataFrame(membership_score).reset_index()
membership_score.columns = ['멤버십종류', 'count']
membership_score['score'] = 0

In [ ]:
score = {'무료':1, '노블':2, '싹틔우미':2, '그린':4, '블루':6, '골드':12}

for i in range(membership_score.shape[0]):
    if pd.isna(membership_score.멤버십종류[i]):
        s = 0
    else:
        arr = membership_score.멤버십종류[i].split(', ')
        s = 0
        for j in arr:
            if j in score:
                s += score.get(j)
        membership_score.loc[i, 'score'] += s

In [ ]:
membership_score.sort_values('score', ascending=False, inplace=True)
membership_score.reset_index(drop=True, inplace=True)
membership_score

,멤버십종류,count,score
0,"무료, 노블, 골드, 블루, 그린",82,25
1,"무료, 싹틔우미, 골드, 블루, 그린",778,25
2,"무료, 골드, 블루, 그린",10097,23
3,"무료, 노블, 골드, 블루",1775,21
4,"무료, 싹틔우미, 골드, 블루",1322,21
5,"무료, 골드, 블루",30009,19
6,"무료, 싹틔우미, 골드, 그린",1297,19
7,"무료, 노블, 골드, 그린",1624,19
8,"무료, 골드, 그린",17967,17
9,"무료, 노블, 골드",2862,15


### 멤버십등급 분류 기준
- S (21~25) : 유료회원(골드, 블루, 그린) 모두 가입 (10,957)

- A (17~19) : 유료회원 2개 이상(골드 포함) 가입 (53,994)
- B (13~15) : 골드 1개 or 싹틔우미/노블 + 블루 + 그린 (75,961)
- C (5~11) : 무료, 싹틔우미/노블, 블루, 그린 (105,605)
- D (1~4) : 무료회원 or 무료 + 싹틔우미/노블 (105,136)
- E (0) : 비회원 (442,260)

In [ ]:
def score_to_grade(s):
    if s in range(21, 26): grade = 'S'
    elif s in range(17, 20): grade = 'A'
    elif s in range(13, 16): grade = 'B'
    elif s in range(5, 12): grade = 'C'
    elif s in range(1, 5): grade = 'D'
    elif s == 0: grade = 'E'

    return grade

membership_score['grade'] = membership_score.score.apply(score_to_grade)

In [ ]:
membership_score.columns = ['멤버십', 'count', '멤버십점수', '멤버십등급']
membership_score

,멤버십,count,멤버십점수,멤버십등급
0,"무료, 노블, 골드, 블루, 그린",82,25,S
1,"무료, 싹틔우미, 골드, 블루, 그린",778,25,S
2,"무료, 골드, 블루, 그린",10097,23,S
3,"무료, 노블, 골드, 블루",1775,21,S
4,"무료, 싹틔우미, 골드, 블루",1322,21,S
5,"무료, 골드, 블루",30009,19,A
6,"무료, 싹틔우미, 골드, 그린",1297,19,A
7,"무료, 노블, 골드, 그린",1624,19,A
8,"무료, 골드, 그린",17967,17,A
9,"무료, 노블, 골드",2862,15,B


In [ ]:
df.drop('멤버십점수', inplace=True, axis=1)

In [ ]:
df.멤버십.value_counts(dropna=False)

NaN                     442260
무료, 골드                   69274
무료                       67162
무료, 그린                   48496
무료, 싹틔우미                 34787
무료, 골드, 블루               30009
무료, 블루                   25712
무료, 골드, 그린               17967
무료, 블루, 그린               14562
무료, 싹틔우미, 그린             10831
무료, 골드, 블루, 그린           10097
무료, 노블                    3187
무료, 싹틔우미, 블루              2911
무료, 노블, 골드                2862
무료, 싹틔우미, 골드              2214
무료, 노블, 그린                1999
무료, 노블, 골드, 블루            1775
무료, 노블, 골드, 그린            1624
무료, 싹틔우미, 블루, 그린          1367
무료, 싹틔우미, 골드, 블루          1322
무료, 싹틔우미, 골드, 그린          1297
무료, 노블, 블루                1094
무료, 싹틔우미, 골드, 블루, 그린       778
무료, 노블, 블루, 그린             244
무료, 노블, 골드, 블루, 그린          82
Name: 멤버십, dtype: int64

In [ ]:
df = pd.merge(df, membership_score, on='멤버십', how='left')
df.drop('count', inplace=True, axis=1)

In [ ]:
df.columns

Index(['연령', '성별', '예매일자', '예매시간', '공연일자', '공연시작시간', '좌석', '결제가격', '취소여부',
       '할인내역', '선예매시작일', '예매시작일', '장르', '휴게시간', '회원여부', '공연명', '내한여부', '출연진',
       '제작진', '러닝타임', '관람연령', '제작사', '가격종류', 'R석', 'S석', 'A석', '티켓판매량', '요일',
       '멤버십', '무료', '싹틔우미', '노블', '골드', '블루', '그린', '층', '블록', '열', '좌석번호',
       '멤버십점수', '멤버십등급'],
      dtype='object')

In [ ]:
df.멤버십등급.value_counts()

E    442260
C    105605
D    105136
B     75961
A     50897
S     14054
Name: 멤버십등급, dtype: int64

In [ ]:
df.to_csv('./data/final_data_membership.csv', index=False)